导入需要使用的库

In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

配置超参数

In [5]:
input_size = 784 # 输入的尺寸，使用MNIST还是 28*28*1的灰度图片
hidden_size = 500 # 隐藏层的大小
num_classes = 10 # 输出层大小
num_epochs = 5
batch_size = 100
learning_rate = 0.001

由于这个训练的模型比前面的大，如果有GPU就使用GPU训练，需要使用到：

In [23]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


创建数据集：

In [9]:
# 获取数据集
train_dataset=torchvision.datasets.MNIST(root='../../data', # 文件夹位置
                                         train=True, # 是否为训练数据
                                         transform=transforms.ToTensor(), # 转化为Tensor
                                         download=True # 下载数据集
                                        )
test_dataset=torchvision.datasets.MNIST(root='../../data', # 文件夹位置
                                         train=False, # 是否为训练数据
                                         transform=transforms.ToTensor(), # 转化为Tensor
                                         download=True # 下载数据集
                                        )

# 构建DataLoader
train_Loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True # 表示打乱顺序
                                        )
test_Loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=True # 表示打乱顺序
                                       )

通过继承`nn.Module`来创建模型：

In [17]:
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        """
        构造函数，需要输入必要的参数,并创建对应的层
        """
        super(Net,self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,output_size)
    def forward(self,x):
        """
        构建计算图
        """
        out=self.fc1(x)
        out=self.relu(out)
        out=self.fc2(out)
        return out

创建一个Net的实例，同样需要放在GPU上。

In [28]:
net=Net(input_size,hidden_size,num_classes).to(device)

构建优化器与损失函数

In [29]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=learning_rate)

训练模型：

In [30]:
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_Loader):
        images=images.reshape(-1,28*28).to(device) # 使用GPU训练
        labels=labels.to(device)
        
        # 前向计算输出值
        outputs=net(images)
        # 计算loss值
        loss=criterion(outputs,labels)
        # 反向计算
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, loss.item()))
        

Epoch [1/5], Step [100], Loss: 0.2008
Epoch [1/5], Step [200], Loss: 0.2311
Epoch [1/5], Step [300], Loss: 0.2119
Epoch [1/5], Step [400], Loss: 0.2388
Epoch [1/5], Step [500], Loss: 0.0688
Epoch [1/5], Step [600], Loss: 0.1361
Epoch [2/5], Step [100], Loss: 0.1514
Epoch [2/5], Step [200], Loss: 0.1207
Epoch [2/5], Step [300], Loss: 0.0711
Epoch [2/5], Step [400], Loss: 0.1649
Epoch [2/5], Step [500], Loss: 0.0496
Epoch [2/5], Step [600], Loss: 0.1133
Epoch [3/5], Step [100], Loss: 0.0604
Epoch [3/5], Step [200], Loss: 0.0503
Epoch [3/5], Step [300], Loss: 0.0263
Epoch [3/5], Step [400], Loss: 0.0183
Epoch [3/5], Step [500], Loss: 0.0439
Epoch [3/5], Step [600], Loss: 0.0465
Epoch [4/5], Step [100], Loss: 0.0616
Epoch [4/5], Step [200], Loss: 0.0449
Epoch [4/5], Step [300], Loss: 0.0643
Epoch [4/5], Step [400], Loss: 0.0746
Epoch [4/5], Step [500], Loss: 0.0539
Epoch [4/5], Step [600], Loss: 0.1150
Epoch [5/5], Step [100], Loss: 0.0459
Epoch [5/5], Step [200], Loss: 0.0234
Epoch [5/5],

测试模型：

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_Loader():
        images=images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        
        output=net(images)
        _,predicted=torch.max(output.data,1)
        total+=labels.size(0)
        correct += (predicted == labels).sum().item()
        